In [ ]:

!apt-get update
!apt-get install -y wget unzip libvulkan1

!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get install -f -y

!apt-get install -y xvfb

!pip install selenium chromedriver-autoinstaller

import chromedriver_autoinstaller
chromedriver_autoinstaller.install()

from selenium import webdriver
from selenium.webdriver.chrome.options import Options


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,031 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubun

In [ ]:
from datetime import date
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time
from bs4 import BeautifulSoup
import chromedriver_autoinstaller
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from google.colab import files

chromedriver_autoinstaller.install()

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# WebDriver
driver = webdriver.Chrome(options=chrome_options)

# testing with limit in the beginning as to test. in the end, just set limit to more than i know information has
article_limit = 5000
scraped_articles_count = 0
checkpoint_interval = 25  # saving at every 25 articles
print_interval = 10

last_checkpoint_count = 0

# Login credentials
email = "grp613@alumni.ku.dk"
password = "xxxx"

# accept cookies
def accept_cookies():
    try:
        accept_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'button#acceptButton'))
        )
        accept_button.click()
    except Exception as e:
        pass

# handle login
def login():
    try:
        driver.get("https://www.information.dk/user/login")
        accept_cookies()
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "edit-name")))
        driver.find_element(By.ID, "edit-name").send_keys(email)
        driver.find_element(By.ID, "edit-pass").send_keys(password)
        driver.find_element(By.ID, "edit-submit").click()
        accept_cookies()
    except Exception as e:
        pass

# paywall by logging in again
def handle_paywall(article_url):
    try:
        login()
        driver.get(article_url)
        time.sleep(3)
        accept_cookies()
    except Exception as e:
        pass

# handle popup if appears
def close_popup():
    try:
        close_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'svg.PopupCloseButton__PopupCloseIcon'))
        )
        close_button.click()
    except Exception as e:
        pass

# getting content
def scrape_article(article_url):
    global scraped_articles_count
    if scraped_articles_count >= article_limit:
        return None

    try:
        driver.get(article_url)
        time.sleep(2)
        close_popup()
        accept_cookies()

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

        if driver.find_elements(By.CSS_SELECTOR, 'div#paywall.premium--box'):
            handle_paywall(article_url)
            time.sleep(2)

        header = driver.find_element(By.TAG_NAME, 'h1').text if driver.find_elements(By.TAG_NAME, 'h1') else 'No header'
        subheader = driver.find_element(By.CSS_SELECTOR, '.typo__subtitle').text if driver.find_elements(By.CSS_SELECTOR, '.typo__subtitle') else 'No subheader'
        category = driver.find_element(By.CSS_SELECTOR, '.typo__desk').text if driver.find_elements(By.CSS_SELECTOR, '.typo__desk') else 'No category'
        date_element = driver.find_element(By.CSS_SELECTOR, '.typo__publication-date time') if driver.find_elements(By.CSS_SELECTOR, '.typo__publication-date time') else None
        date = date_element.get_attribute('datetime') if date_element else 'No date'

        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.typo__body')))
        content_div = driver.find_element(By.CSS_SELECTOR, 'div.typo__body')
        content = driver.execute_script("return arguments[0].innerText;", content_div)

        if not content.strip():
            paragraphs = driver.find_elements(By.CSS_SELECTOR, 'div.typo__body p')
            content = ' '.join([p.text for p in paragraphs]) if paragraphs else 'No content'

        scraped_articles_count += 1

        if scraped_articles_count % print_interval == 0:
            print(f"Scraped {scraped_articles_count} articles so far...")

        return {'URL': article_url, 'Header': header, 'Subheader': subheader, 'Category': category, 'Content': content, 'Date': date, 'Search term': keyword}

    except Exception as e:
        return None

def scrape_page(page_url):
    try:
        driver.get(page_url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        articles = soup.select('div.search_content_results h2 a')
        article_urls = ['https://www.information.dk' + article.get('href') for article in articles]
        page_articles = []

        for article_url in article_urls:
            if scraped_articles_count >= article_limit:
                break
            data = scrape_article(article_url)
            if data:
                page_articles.append(data)

        return page_articles

    except Exception as e:
        return []

base_url = 'https://www.information.dk/search/articles?search_api_fulltext={}&page={}'

# Keywords
keywords = ['facial recognition', 'face recognition', 'ansigtsaftryk', 'ansigtsscan',
                'masseovervågning ansigt', 'masseovervågning kamera', 'ansigtskamera', 'overvågningssamfund ansigt',
                'overvågningssamfund kamera', 'sikkerhedspakke overvågning', 'tryghedsskabende kamera']

unique_articles = []
checkpoint_index = 0

def save_checkpoint(new_articles, checkpoint_index):
    df_checkpoint = pd.DataFrame(new_articles)
    checkpoint_filename = f'articles_checkpoint_{checkpoint_index}.csv'
    df_checkpoint.to_csv(checkpoint_filename, index=False)
    files.download(checkpoint_filename)
    print(f"Saved checkpoint at article {scraped_articles_count} with {len(new_articles)} new articles.")

login()

for keyword in keywords:
    current_page = 0

    while scraped_articles_count < article_limit:
        page_url = base_url.format(keyword, current_page)

        articles_on_page = scrape_page(page_url)

        if not articles_on_page:
            break

        new_articles = []
        for article in articles_on_page:
            if article['URL'] not in [art['URL'] for art in unique_articles]:
                unique_articles.append(article)
                new_articles.append(article)

        # Save at every 25 article
        if scraped_articles_count >= last_checkpoint_count + checkpoint_interval:
            last_checkpoint_count = scraped_articles_count
            save_checkpoint(unique_articles, checkpoint_index)
            checkpoint_index += 1

        current_page += 1

df = pd.DataFrame(unique_articles)
df.to_csv('Information_ansigtsgenkendelse.csv', index=False)
files.download('Information_ansigtsgenkendelse.csv')

driver.quit()


Scraped 10 articles so far...
Scraped 20 articles so far...
Scraped 30 articles so far...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Saved checkpoint at article 32 with 32 new articles.
Scraped 40 articles so far...
Scraped 50 articles so far...
Scraped 60 articles so far...


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Saved checkpoint at article 165 with 150 new articles.
Scraped 170 articles so far...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df

,URL,Header,Subheader,Category,Content,Date,Search term
0,https://www.information.dk/kultur/2014/01/kund...,Kundeklubber får forbrugere til at sælge ud af...,"Når butikkerne lokker med medlemskort, handler...",KULTUR,»Så er du klar.«\n\nEkspedienten smiler og ræk...,2014-01-13T13:00:00Z,facial recognition
1,https://www.information.dk/nyhedsblog/2011/09/...,Det tredje køn er fundet: X,"Transseksuelle og personer, der skifter køn ha...",INDLAND,"Transseksuelle og personer, der skifter køn ha...",2011-09-15T16:11:32Z,face recognition
2,https://www.information.dk/martin-burcharths-b...,"""Lovefest med John McCain""",På rundtur i Europa har præsident Zelenskyj fo...,INDLAND,Barack Obamas rådgivere fik for en tiden siden...,2009-01-20T04:59:41Z,face recognition
3,https://www.information.dk/martin-burcharths-b...,"Hillary, Kennedy og kvindelige vælgere",På rundtur i Europa har præsident Zelenskyj fo...,INDLAND,Det var interessant at læse sidste søndag en e...,2008-05-28T17:54:44Z,face recognition
4,https://www.information.dk/martin-burcharths-b...,Præsidentkandidaternes økonomiske idéer,På rundtur i Europa har præsident Zelenskyj fo...,INDLAND,I forbindelse med dagens forsideartikel i Info...,2008-01-22T23:16:35Z,face recognition
...,...,...,...,...,...,...,...
155,https://www.information.dk/moti/2007/06/neurot...,Den neurotiske kontrolfeber,Frygtens kultur. Hvis overvågning skaber trygh...,MODERNE TIDER,"C.F. Tietgen, der opbyggede Store Nordiske Tel...",2007-06-09T14:00:00Z,tryghedsskabende kamera
156,https://www.information.dk/indland/2019/10/age...,"Agenter på nettet, grænsekontrol og overvågnin...","Vi har bedt Jacob Mchangama, direktør for tænk...",INDLAND,For at øge befolkningens tryghed vil justitsmi...,2019-10-11T14:00:00Z,tryghedsskabende kamera
157,https://www.information.dk/1999/01/mad,Mad,Engang var en kvindelig departementschef et sæ...,No category,Nu hvor TV 2's morgenstudie har fået installer...,1999-01-16T13:00:00Z,tryghedsskabende kamera
158,https://www.information.dk/debat/2018/05/gymna...,"Gymnasielærer: Det er os selv, der skaber fryg...",Vådeskuddet på betjenten ved Københavns Synago...,DEBAT,For et par uger siden blev en politibetjent ra...,2018-05-22T14:00:00Z,tryghedsskabende kamera


In [ ]:
df['merged_text'] = df['Header'].fillna('') + ' ' + df['Subheader'].fillna('') + ' ' + df['Content'].fillna('')
df


,URL,Header,Subheader,Category,Content,Date,Search term,merged_text
0,https://www.information.dk/kultur/2014/01/kund...,Kundeklubber får forbrugere til at sælge ud af...,"Når butikkerne lokker med medlemskort, handler...",KULTUR,»Så er du klar.«\n\nEkspedienten smiler og ræk...,2014-01-13T13:00:00Z,facial recognition,Kundeklubber får forbrugere til at sælge ud af...
1,https://www.information.dk/nyhedsblog/2011/09/...,Det tredje køn er fundet: X,"Transseksuelle og personer, der skifter køn ha...",INDLAND,"Transseksuelle og personer, der skifter køn ha...",2011-09-15T16:11:32Z,face recognition,Det tredje køn er fundet: X Transseksuelle og ...
2,https://www.information.dk/martin-burcharths-b...,"""Lovefest med John McCain""",På rundtur i Europa har præsident Zelenskyj fo...,INDLAND,Barack Obamas rådgivere fik for en tiden siden...,2009-01-20T04:59:41Z,face recognition,"""Lovefest med John McCain"" På rundtur i Europa..."
3,https://www.information.dk/martin-burcharths-b...,"Hillary, Kennedy og kvindelige vælgere",På rundtur i Europa har præsident Zelenskyj fo...,INDLAND,Det var interessant at læse sidste søndag en e...,2008-05-28T17:54:44Z,face recognition,"Hillary, Kennedy og kvindelige vælgere På rund..."
4,https://www.information.dk/martin-burcharths-b...,Præsidentkandidaternes økonomiske idéer,På rundtur i Europa har præsident Zelenskyj fo...,INDLAND,I forbindelse med dagens forsideartikel i Info...,2008-01-22T23:16:35Z,face recognition,Præsidentkandidaternes økonomiske idéer På run...
...,...,...,...,...,...,...,...,...
155,https://www.information.dk/moti/2007/06/neurot...,Den neurotiske kontrolfeber,Frygtens kultur. Hvis overvågning skaber trygh...,MODERNE TIDER,"C.F. Tietgen, der opbyggede Store Nordiske Tel...",2007-06-09T14:00:00Z,tryghedsskabende kamera,Den neurotiske kontrolfeber Frygtens kultur. H...
156,https://www.information.dk/indland/2019/10/age...,"Agenter på nettet, grænsekontrol og overvågnin...","Vi har bedt Jacob Mchangama, direktør for tænk...",INDLAND,For at øge befolkningens tryghed vil justitsmi...,2019-10-11T14:00:00Z,tryghedsskabende kamera,"Agenter på nettet, grænsekontrol og overvågnin..."
157,https://www.information.dk/1999/01/mad,Mad,Engang var en kvindelig departementschef et sæ...,No category,Nu hvor TV 2's morgenstudie har fået installer...,1999-01-16T13:00:00Z,tryghedsskabende kamera,Mad Engang var en kvindelig departementschef e...
158,https://www.information.dk/debat/2018/05/gymna...,"Gymnasielærer: Det er os selv, der skaber fryg...",Vådeskuddet på betjenten ved Københavns Synago...,DEBAT,For et par uger siden blev en politibetjent ra...,2018-05-22T14:00:00Z,tryghedsskabende kamera,"Gymnasielærer: Det er os selv, der skaber fryg..."


In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/Thesis/Data/Information/Information_ansigtsgenkendelse-5.csv')
df1 = pd.read_csv('/content/drive/MyDrive/Thesis/Data/Information/Information_ansigtsgenkendelse.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
merged_df = pd.concat([df, df1], ignore_index=True)
merged_df

,URL,Header,Subheader,Category,Content,Date,Search term
0,https://www.information.dk/indland/leder/2020/...,"Hvis Venstre igen vil kaldes liberalt, kunne p...",Ligesom Liberal Alliance kunne Venstre have sa...,INDLAND,Endnu en gang har Venstre forpasset chancen fo...,2020-02-04T13:00:00Z,ansigtsgenkendelse
1,https://www.information.dk/telegram/2019/09/mi...,Microsoft frygter misbrug af ansigtsgenkendelse,"Microsoft siger, at selskabet aldrig vil sælge...",No category,"Microsoft har afvist ønsker fra regeringer, de...",2019-09-13T18:23:03Z,ansigtsgenkendelse
2,https://www.information.dk/telegram/2020/02/ve...,Venstre går nu officielt ind for brug af ansig...,Inger Støjbergs forslag om at bruge ansigtsgen...,No category,Venstre går nu officielt ind for brug af ansig...,2020-02-03T00:49:03Z,ansigtsgenkendelse
3,https://www.information.dk/telegram/2021/11/fa...,Facebook dropper ansigtsgenkendelse på foto og...,"En milliard folks ""skabeloner"" til ansigtsgenk...",No category,Det sociale medie Facebook vil stoppe med at a...,2021-11-02T18:39:04Z,ansigtsgenkendelse
4,https://www.information.dk/telegram/2020/01/al...,Alternativet om overvågning: Danmark skal ikke...,Folketinget skal tirsdag drøfte Alternativets ...,No category,"De er klar over, at beslutningsforslaget næppe...",2020-01-21T04:32:03Z,ansigtsgenkendelse
...,...,...,...,...,...,...,...
238,https://www.information.dk/2019/11/maaske-begy...,Måske skal vi begynde at se det sociale som de...,"Kunstig intelligens er i udvikling i disse år,...",REDAKTIONEN,Robotterne kommer til at overtage vores job og...,NaN,ansigtsgenkendelse
239,https://www.information.dk/indland/2024/02/nic...,Nick Hækkerup: »Det er rart at komme ud i en v...,"Politikere og presse spiller hinanden dårlige,...",INDLAND,"»Politik drejer sig om en eneste ting, og det ...",NaN,ansigtsgenkendelse
240,https://www.information.dk/kultur/2019/06/fejl...,"Det er fejlene, der gør os menneskelige. Så hv...","Det er igennem vores fejl, at vi udvikler os, ...",KULTUR,For to måneder siden stod jeg foran en hæveaut...,NaN,ansigtsgenkendelse
241,https://www.information.dk/udland/2020/05/naom...,Naomi Klein: Sådan planlægger techindustrien a...,"Mens coronavirussen dræber, kører techgiganter...",UDLAND,Onsdag den 6. maj var New York-guvernør Andrew...,NaN,ansigtsgenkendelse


In [ ]:
merged_df.drop_duplicates(subset=['URL'], inplace=True)
merged_df.columns = merged_df.columns.str.lower()


In [ ]:
# fixing date
df['cleaned_date'] = df['Date'].str[:10]

df['date'] = pd.to_datetime(df['cleaned_date'], format='%Y-%m-%d')



,URL,Header,Subheader,Category,Content,Date,Search term,merged_text,cleaned_date,date
0,https://www.information.dk/kultur/2014/01/kund...,Kundeklubber får forbrugere til at sælge ud af...,"Når butikkerne lokker med medlemskort, handler...",KULTUR,»Så er du klar.«\n\nEkspedienten smiler og ræk...,2014-01-13T13:00:00Z,facial recognition,Kundeklubber får forbrugere til at sælge ud af...,2014-01-13,2014-01-13
1,https://www.information.dk/nyhedsblog/2011/09/...,Det tredje køn er fundet: X,"Transseksuelle og personer, der skifter køn ha...",INDLAND,"Transseksuelle og personer, der skifter køn ha...",2011-09-15T16:11:32Z,face recognition,Det tredje køn er fundet: X Transseksuelle og ...,2011-09-15,2011-09-15
2,https://www.information.dk/martin-burcharths-b...,"""Lovefest med John McCain""",På rundtur i Europa har præsident Zelenskyj fo...,INDLAND,Barack Obamas rådgivere fik for en tiden siden...,2009-01-20T04:59:41Z,face recognition,"""Lovefest med John McCain"" På rundtur i Europa...",2009-01-20,2009-01-20
3,https://www.information.dk/martin-burcharths-b...,"Hillary, Kennedy og kvindelige vælgere",På rundtur i Europa har præsident Zelenskyj fo...,INDLAND,Det var interessant at læse sidste søndag en e...,2008-05-28T17:54:44Z,face recognition,"Hillary, Kennedy og kvindelige vælgere På rund...",2008-05-28,2008-05-28
4,https://www.information.dk/martin-burcharths-b...,Præsidentkandidaternes økonomiske idéer,På rundtur i Europa har præsident Zelenskyj fo...,INDLAND,I forbindelse med dagens forsideartikel i Info...,2008-01-22T23:16:35Z,face recognition,Præsidentkandidaternes økonomiske idéer På run...,2008-01-22,2008-01-22
...,...,...,...,...,...,...,...,...,...,...
155,https://www.information.dk/moti/2007/06/neurot...,Den neurotiske kontrolfeber,Frygtens kultur. Hvis overvågning skaber trygh...,MODERNE TIDER,"C.F. Tietgen, der opbyggede Store Nordiske Tel...",2007-06-09T14:00:00Z,tryghedsskabende kamera,Den neurotiske kontrolfeber Frygtens kultur. H...,2007-06-09,2007-06-09
156,https://www.information.dk/indland/2019/10/age...,"Agenter på nettet, grænsekontrol og overvågnin...","Vi har bedt Jacob Mchangama, direktør for tænk...",INDLAND,For at øge befolkningens tryghed vil justitsmi...,2019-10-11T14:00:00Z,tryghedsskabende kamera,"Agenter på nettet, grænsekontrol og overvågnin...",2019-10-11,2019-10-11
157,https://www.information.dk/1999/01/mad,Mad,Engang var en kvindelig departementschef et sæ...,No category,Nu hvor TV 2's morgenstudie har fået installer...,1999-01-16T13:00:00Z,tryghedsskabende kamera,Mad Engang var en kvindelig departementschef e...,1999-01-16,1999-01-16
158,https://www.information.dk/debat/2018/05/gymna...,"Gymnasielærer: Det er os selv, der skaber fryg...",Vådeskuddet på betjenten ved Københavns Synago...,DEBAT,For et par uger siden blev en politibetjent ra...,2018-05-22T14:00:00Z,tryghedsskabende kamera,"Gymnasielærer: Det er os selv, der skaber fryg...",2018-05-22,2018-05-22


In [ ]:
df.drop(columns=['cleaned_date', 'Date'], inplace=True)
df

,URL,Header,Subheader,Category,Content,Search term,merged_text,date
0,https://www.information.dk/kultur/2014/01/kund...,Kundeklubber får forbrugere til at sælge ud af...,"Når butikkerne lokker med medlemskort, handler...",KULTUR,»Så er du klar.«\n\nEkspedienten smiler og ræk...,facial recognition,Kundeklubber får forbrugere til at sælge ud af...,2014-01-13
1,https://www.information.dk/nyhedsblog/2011/09/...,Det tredje køn er fundet: X,"Transseksuelle og personer, der skifter køn ha...",INDLAND,"Transseksuelle og personer, der skifter køn ha...",face recognition,Det tredje køn er fundet: X Transseksuelle og ...,2011-09-15
2,https://www.information.dk/martin-burcharths-b...,"""Lovefest med John McCain""",På rundtur i Europa har præsident Zelenskyj fo...,INDLAND,Barack Obamas rådgivere fik for en tiden siden...,face recognition,"""Lovefest med John McCain"" På rundtur i Europa...",2009-01-20
3,https://www.information.dk/martin-burcharths-b...,"Hillary, Kennedy og kvindelige vælgere",På rundtur i Europa har præsident Zelenskyj fo...,INDLAND,Det var interessant at læse sidste søndag en e...,face recognition,"Hillary, Kennedy og kvindelige vælgere På rund...",2008-05-28
4,https://www.information.dk/martin-burcharths-b...,Præsidentkandidaternes økonomiske idéer,På rundtur i Europa har præsident Zelenskyj fo...,INDLAND,I forbindelse med dagens forsideartikel i Info...,face recognition,Præsidentkandidaternes økonomiske idéer På run...,2008-01-22
...,...,...,...,...,...,...,...,...
155,https://www.information.dk/moti/2007/06/neurot...,Den neurotiske kontrolfeber,Frygtens kultur. Hvis overvågning skaber trygh...,MODERNE TIDER,"C.F. Tietgen, der opbyggede Store Nordiske Tel...",tryghedsskabende kamera,Den neurotiske kontrolfeber Frygtens kultur. H...,2007-06-09
156,https://www.information.dk/indland/2019/10/age...,"Agenter på nettet, grænsekontrol og overvågnin...","Vi har bedt Jacob Mchangama, direktør for tænk...",INDLAND,For at øge befolkningens tryghed vil justitsmi...,tryghedsskabende kamera,"Agenter på nettet, grænsekontrol og overvågnin...",2019-10-11
157,https://www.information.dk/1999/01/mad,Mad,Engang var en kvindelig departementschef et sæ...,No category,Nu hvor TV 2's morgenstudie har fået installer...,tryghedsskabende kamera,Mad Engang var en kvindelig departementschef e...,1999-01-16
158,https://www.information.dk/debat/2018/05/gymna...,"Gymnasielærer: Det er os selv, der skaber fryg...",Vådeskuddet på betjenten ved Københavns Synago...,DEBAT,For et par uger siden blev en politibetjent ra...,tryghedsskabende kamera,"Gymnasielærer: Det er os selv, der skaber fryg...",2018-05-22


In [ ]:
df.columns = df.columns.str.lower()
df

,url,header,subheader,category,content,search term,merged_text,date
0,https://www.information.dk/kultur/2014/01/kund...,Kundeklubber får forbrugere til at sælge ud af...,"Når butikkerne lokker med medlemskort, handler...",KULTUR,»Så er du klar.«\n\nEkspedienten smiler og ræk...,facial recognition,Kundeklubber får forbrugere til at sælge ud af...,2014-01-13
1,https://www.information.dk/nyhedsblog/2011/09/...,Det tredje køn er fundet: X,"Transseksuelle og personer, der skifter køn ha...",INDLAND,"Transseksuelle og personer, der skifter køn ha...",face recognition,Det tredje køn er fundet: X Transseksuelle og ...,2011-09-15
2,https://www.information.dk/martin-burcharths-b...,"""Lovefest med John McCain""",På rundtur i Europa har præsident Zelenskyj fo...,INDLAND,Barack Obamas rådgivere fik for en tiden siden...,face recognition,"""Lovefest med John McCain"" På rundtur i Europa...",2009-01-20
3,https://www.information.dk/martin-burcharths-b...,"Hillary, Kennedy og kvindelige vælgere",På rundtur i Europa har præsident Zelenskyj fo...,INDLAND,Det var interessant at læse sidste søndag en e...,face recognition,"Hillary, Kennedy og kvindelige vælgere På rund...",2008-05-28
4,https://www.information.dk/martin-burcharths-b...,Præsidentkandidaternes økonomiske idéer,På rundtur i Europa har præsident Zelenskyj fo...,INDLAND,I forbindelse med dagens forsideartikel i Info...,face recognition,Præsidentkandidaternes økonomiske idéer På run...,2008-01-22
...,...,...,...,...,...,...,...,...
155,https://www.information.dk/moti/2007/06/neurot...,Den neurotiske kontrolfeber,Frygtens kultur. Hvis overvågning skaber trygh...,MODERNE TIDER,"C.F. Tietgen, der opbyggede Store Nordiske Tel...",tryghedsskabende kamera,Den neurotiske kontrolfeber Frygtens kultur. H...,2007-06-09
156,https://www.information.dk/indland/2019/10/age...,"Agenter på nettet, grænsekontrol og overvågnin...","Vi har bedt Jacob Mchangama, direktør for tænk...",INDLAND,For at øge befolkningens tryghed vil justitsmi...,tryghedsskabende kamera,"Agenter på nettet, grænsekontrol og overvågnin...",2019-10-11
157,https://www.information.dk/1999/01/mad,Mad,Engang var en kvindelig departementschef et sæ...,No category,Nu hvor TV 2's morgenstudie har fået installer...,tryghedsskabende kamera,Mad Engang var en kvindelig departementschef e...,1999-01-16
158,https://www.information.dk/debat/2018/05/gymna...,"Gymnasielærer: Det er os selv, der skaber fryg...",Vådeskuddet på betjenten ved Københavns Synago...,DEBAT,For et par uger siden blev en politibetjent ra...,tryghedsskabende kamera,"Gymnasielærer: Det er os selv, der skaber fryg...",2018-05-22


In [ ]:
merged_df['merged_content'] = merged_df['header'] + ' ' + merged_df['subheader'] + ' ' + merged_df['content']
merged_df

,url,header,subheader,category,content,date,search term,merged_content
0,https://www.information.dk/indland/leder/2020/...,"Hvis Venstre igen vil kaldes liberalt, kunne p...",Ligesom Liberal Alliance kunne Venstre have sa...,INDLAND,Endnu en gang har Venstre forpasset chancen fo...,2020-02-04,ansigtsgenkendelse,"Hvis Venstre igen vil kaldes liberalt, kunne p..."
1,https://www.information.dk/telegram/2019/09/mi...,Microsoft frygter misbrug af ansigtsgenkendelse,"Microsoft siger, at selskabet aldrig vil sælge...",No category,"Microsoft har afvist ønsker fra regeringer, de...",2019-09-13,ansigtsgenkendelse,Microsoft frygter misbrug af ansigtsgenkendels...
2,https://www.information.dk/telegram/2020/02/ve...,Venstre går nu officielt ind for brug af ansig...,Inger Støjbergs forslag om at bruge ansigtsgen...,No category,Venstre går nu officielt ind for brug af ansig...,2020-02-03,ansigtsgenkendelse,Venstre går nu officielt ind for brug af ansig...
3,https://www.information.dk/telegram/2021/11/fa...,Facebook dropper ansigtsgenkendelse på foto og...,"En milliard folks ""skabeloner"" til ansigtsgenk...",No category,Det sociale medie Facebook vil stoppe med at a...,2021-11-02,ansigtsgenkendelse,Facebook dropper ansigtsgenkendelse på foto og...
4,https://www.information.dk/telegram/2020/01/al...,Alternativet om overvågning: Danmark skal ikke...,Folketinget skal tirsdag drøfte Alternativets ...,No category,"De er klar over, at beslutningsforslaget næppe...",2020-01-21,ansigtsgenkendelse,Alternativet om overvågning: Danmark skal ikke...
...,...,...,...,...,...,...,...,...
118,https://www.information.dk/udland/2020/05/naom...,Naomi Klein: Sådan planlægger techindustrien a...,"Mens coronavirussen dræber, kører techgiganter...",UDLAND,Onsdag den 6. maj var New York-guvernør Andrew...,2020-05-22,ansigtsgenkendelse,Naomi Klein: Sådan planlægger techindustrien a...
119,https://www.information.dk/indland/2023/04/ryd...,Vi fulgte sporene efter 15 formodede russiske ...,"En lukket by fra Den Kolde Krig, en gul murste...",INDLAND,»Optager du?«\n\nEn kvindestemme svarer ja. Hu...,2023-04-15,ansigtsgenkendelse,Vi fulgte sporene efter 15 formodede russiske ...
120,https://www.information.dk/anders-haahr-rasmus...,"Når køn er noget, man skal lære at se",Risikoen for en ny Trump-regering virker så sk...,INDLAND,En historie i P1 Morgen i morges: Blinde kan f...,2012-01-25,face detect,"Når køn er noget, man skal lære at se Risikoen..."
121,https://www.information.dk/mofo/he-should-been...,He should have been a boy,"He was born Linda, but ten years ago Daniel ca...",MODERNE TIDER,"From the window of his room on the 5th floor, ...",2021-01-12,face detect,"He should have been a boy He was born Linda, b..."


In [ ]:

from google.colab import files

df.to_csv('Information_scraped_articles_clean_all.csv', index=False)

files.download('Information_scraped_articles_clean_all.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd

df = pd.read_csv('/content/gdrive/MyDrive/Thesis/Data/Information/Information_scraped_articles_clean_all.csv')
df1 = pd.read_csv('/content/gdrive/MyDrive/Thesis/Data/Information/Information_scraped_articles_clean.csv')

In [ ]:
df.rename(columns={'merged_text': 'merged_content'}, inplace=True)

In [ ]:
df = pd.concat([df, df1], ignore_index=True)
df

,url,header,subheader,category,content,search term,merged_content,date
0,https://www.information.dk/kultur/2014/01/kund...,Kundeklubber får forbrugere til at sælge ud af...,"Når butikkerne lokker med medlemskort, handler...",KULTUR,»Så er du klar.«\n\nEkspedienten smiler og ræk...,facial recognition,Kundeklubber får forbrugere til at sælge ud af...,2014-01-13
1,https://www.information.dk/nyhedsblog/2011/09/...,Det tredje køn er fundet: X,"Transseksuelle og personer, der skifter køn ha...",INDLAND,"Transseksuelle og personer, der skifter køn ha...",face recognition,Det tredje køn er fundet: X Transseksuelle og ...,2011-09-15
2,https://www.information.dk/martin-burcharths-b...,"""Lovefest med John McCain""",På rundtur i Europa har præsident Zelenskyj fo...,INDLAND,Barack Obamas rådgivere fik for en tiden siden...,face recognition,"""Lovefest med John McCain"" På rundtur i Europa...",2009-01-20
3,https://www.information.dk/martin-burcharths-b...,"Hillary, Kennedy og kvindelige vælgere",På rundtur i Europa har præsident Zelenskyj fo...,INDLAND,Det var interessant at læse sidste søndag en e...,face recognition,"Hillary, Kennedy og kvindelige vælgere På rund...",2008-05-28
4,https://www.information.dk/martin-burcharths-b...,Præsidentkandidaternes økonomiske idéer,På rundtur i Europa har præsident Zelenskyj fo...,INDLAND,I forbindelse med dagens forsideartikel i Info...,face recognition,Præsidentkandidaternes økonomiske idéer På run...,2008-01-22
...,...,...,...,...,...,...,...,...
278,https://www.information.dk/udland/2020/05/naom...,Naomi Klein: Sådan planlægger techindustrien a...,"Mens coronavirussen dræber, kører techgiganter...",UDLAND,Onsdag den 6. maj var New York-guvernør Andrew...,ansigtsgenkendelse,Naomi Klein: Sådan planlægger techindustrien a...,2020-05-22
279,https://www.information.dk/indland/2023/04/ryd...,Vi fulgte sporene efter 15 formodede russiske ...,"En lukket by fra Den Kolde Krig, en gul murste...",INDLAND,»Optager du?«\n\nEn kvindestemme svarer ja. Hu...,ansigtsgenkendelse,Vi fulgte sporene efter 15 formodede russiske ...,2023-04-15
280,https://www.information.dk/anders-haahr-rasmus...,"Når køn er noget, man skal lære at se",Risikoen for en ny Trump-regering virker så sk...,INDLAND,En historie i P1 Morgen i morges: Blinde kan f...,face detect,"Når køn er noget, man skal lære at se Risikoen...",2012-01-25
281,https://www.information.dk/mofo/he-should-been...,He should have been a boy,"He was born Linda, but ten years ago Daniel ca...",MODERNE TIDER,"From the window of his room on the 5th floor, ...",face detect,"He should have been a boy He was born Linda, b...",2021-01-12


In [ ]:
from google.colab import files

df_filtered.to_csv('Information.csv', index=False)
files.download('Information.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>